# Calculate NPV

## Setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
# Import libraries
import pandas as pd
import numpy as np
from investing.config.file_paths import input_data, processed_data, output_data
from investing.tools.functions import npv_calc, pv_full_output

<IPython.core.display.Javascript object>

## Configure

In [4]:
# File names
parameters_file = "params_dividends.csv"
start_values = "start_data_dividends.csv"
npv_file = "npv_dividends.csv"

<IPython.core.display.Javascript object>

## Calculate NPV

In [5]:
# Load dataframe of all input values
df_params = pd.read_csv(input_data / parameters_file)
df_start = pd.read_csv(processed_data / start_values)

<IPython.core.display.Javascript object>

In [6]:
# Build the dataframe
df = pd.merge(df_params, df_start, "outer", "ticker")

<IPython.core.display.Javascript object>

In [7]:
# Set the Discount Rate
df["discount_rate"] = 10

<IPython.core.display.Javascript object>

In [8]:
# Calculate the NPV
for pvalue in ["p10", "p50", "p90"]:
    starting_value = df["start_value"]
    discount_rate = df["discount_rate"]
    perpetual_rate = df[f"{pvalue}_terminal"]
    growth_stage1 = df[f"{pvalue}_stage1"]
    growth_stage2 = df[f"{pvalue}_stage2"]
    payout_ratio = df["payout_ratio"]

    df[f"npv_{pvalue}"] = np.vectorize(npv_calc)(
        starting_value,
        discount_rate,
        growth_stage1,
        growth_stage2,
        perpetual_rate,
        payout_ratio,
    )

<IPython.core.display.Javascript object>

In [9]:
# Calculate the present value forecasts
pv_forecasts = {}
for index, row in df.iterrows():
    temp_dict = {}

    ticker = row["ticker"]

    for pvalue in ["p10", "p50", "p90"]:
        starting_value = row["start_value"]
        discount_rate = row["discount_rate"]
        growth_stage1 = row[f"{pvalue}_stage1"]
        growth_stage2 = row[f"{pvalue}_stage2"]
        perpetual_rate = row[f"{pvalue}_terminal"]
        payout_ratio = row["payout_ratio"]

        # Set the PV forecast to a temporary dictionary
        temp = pv_full_output(
            starting_value,
            discount_rate,
            growth_stage1,
            growth_stage2,
            perpetual_rate,
            payout_ratio,
        )
        # Extract the PV forecast dataframe from the dictionary and set to the temparory dict
        temp_dict[pvalue] = temp["full_pv"]

    # Set temp dict to the ticker
    pv_forecasts[ticker] = temp_dict

<IPython.core.display.Javascript object>

In [10]:
# Calculate Risked NPV
df["risked_npv"] = 0.3 * df["npv_p10"] + 0.4 * df["npv_p50"] + 0.3 * df["npv_p90"]

<IPython.core.display.Javascript object>

## Export Data

In [11]:
df.to_csv(output_data / npv_file, index=False)

<IPython.core.display.Javascript object>